# PART B

# Imports

In [22]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from scipy.optimize import minimize
from matplotlib import pyplot as plt
from pathlib import Path
from statsmodels.iolib.summary import summary
from tqdm import tqdm
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

# Data

In [23]:
# Read Data - 48 Industry Portfolios Daily
file_path = str(Path().absolute()) + "\Data\\48_Industry_Portfolios_Daily.csv"
df_daily = pd.read_csv(file_path, skiprows=9, nrows=25901, index_col = 0)
df_daily = df_daily/100
df_daily = df_daily.where(df_daily > -0.99, np.nan)


# Read Data - 10 Industry Portfolios Monthly
file_path = str(Path().absolute()) + "\Data\\48_Industry_Portfolios.CSV"
df = pd.read_csv(file_path, skiprows=11, nrows=1182, index_col = 0)
nb_industries = pd.read_csv(file_path, skiprows=2587, nrows=1182, index_col = 0)
avg_size = pd.read_csv(file_path, skiprows=3773, nrows=1182, index_col = 0)
be_me = pd.read_csv(file_path, skiprows=4959, nrows=99, index_col = 0)
df = df/100
df = df.where(df > -0.99, np.nan)
nb_industries = nb_industries.where(nb_industries > -0.99, np.nan)
avg_size = avg_size.where(avg_size > -0.99, np.nan)
be_me = be_me.where(be_me > -0.99, np.nan)


# Read Data - 3 Factor Model Daily (Mkt-RF, SMB, HML)
file_path = str(Path().absolute()) + "\Data\\F-F_Research_Data_Factors_daily.CSV"
fact3_daily = pd.read_csv(file_path, skiprows=4, nrows=25901, index_col = 0)
fact3_daily = fact3_daily/100
fact3_daily = fact3_daily.where(fact3_daily > -0.99, np.nan)


# Read Data - 3 Factor Model Monthly (Mkt-RF, SMB, HML)
file_path = str(Path().absolute()) + "\Data\\F-F_Research_Data_Factors.CSV"
fact3 = pd.read_csv(file_path, skiprows=3, nrows=1182, index_col = 0)
fact3 = fact3/100
fact3 = fact3.where(fact3 > -0.99, np.nan)


# Read Data - MOM Factor Monthly
file_path = str(Path().absolute()) + "\Data\\F-F_Momentum_Factor.CSV"
fact_MOM = pd.read_csv(file_path, skiprows=13, nrows=1176, index_col = 0)
fact_MOM = fact_MOM/100
fact_MOM = fact_MOM.where(fact_MOM > -0.99, np.nan)


# Construct 4 Factor Model
fact4 = pd.concat([fact3, fact_MOM], axis = 1)
fact4 = fact4/100
fact4 = fact4.where(fact4 > -0.99, np.nan)
fact4 = fact4.dropna()



# Read Data - 5 Factor Model Daily (Mkt-RF, SMB, HML, RMW, CMA)
file_path = str(Path().absolute()) + "\Data\\F-F_Research_Data_5_Factors_2x3_daily.CSV"
fact5 = pd.read_csv(file_path, skiprows=3, nrows=15481, index_col = 0)
fact5 = fact5/100
fact5 = fact5.where(fact5 > -0.99, np.nan)


# Convert index column into a date format
df.index = pd.to_datetime(df.index, format ="%Y%m")
nb_industries.index = pd.to_datetime(nb_industries.index, format ="%Y%m")
avg_size.index = pd.to_datetime(avg_size.index, format ="%Y%m")
df_daily.index = pd.to_datetime(df_daily.index, format ="%Y%m%d")
fact3.index = pd.to_datetime(fact3.index, format ="%Y%m")
fact3_daily.index = pd.to_datetime(fact3_daily.index, format ="%Y%m%d")
fact_MOM.index = pd.to_datetime(fact_MOM.index, format ="%Y%m")
fact4.index = pd.to_datetime(fact4.index, format ="%Y%m")
fact5.index = pd.to_datetime(fact5.index, format ="%Y%m%d")
be_me.index = pd.to_datetime(be_me.index, format ="%Y")

# Question 1

In [24]:
# Compute the market capitalization as Average Firm Size x Number of Firms

df_mktcap = avg_size * nb_industries

# Back-fill missing data
df_mktcap.fillna(method='ffill', inplace=True)  # Forward fill missing data

# Question 2

In [92]:
# The ratio of book value (BE: book equity) to market value (ME: market equity), ie book
# to-market ratio, i.e. Book-to-Market ratio, using the "Sum of BE / Sum of ME" data

# Create an empty DataFrame with monthly frequency
monthly_dates = pd.date_range(start=be_me.index.min(), end=be_me.index.max(), freq='MS')
be_me_monthly = pd.DataFrame(index=monthly_dates)


# Forward-fill BE/ME values for each July to June period for each industry
for industry in be_me.columns:
    be_me_industry_monthly = pd.DataFrame(index=monthly_dates)

    for year in be_me.index.year:
        start_date = pd.to_datetime(f"{year}-07-01")
        end_date = pd.to_datetime(f"{year + 1}-06-01")

        if start_date in be_me_industry_monthly.index and end_date in be_me_industry_monthly.index:
            be_me_industry_monthly.loc[start_date:end_date, industry] = be_me.loc[pd.to_datetime(f'{year}-01-01'), industry]


    # Forward-fill any missing values
    be_me_monthly.fillna(method="ffill", inplace=True)

    # Assign the industry data to the main DataFrame
    be_me_monthly[industry] = be_me_industry_monthly[industry]


# Question 3

In [32]:
# The momentum of each industry as the average return for that industry during the last 12 months including month t

momentum = df.rolling(window=12).mean()

# Question 4

In [33]:
# Define the monthly date range
indices = pd.date_range(df_daily.index.min(), df_daily.index.max(), freq="M")

# Ensure df_daily and fact3_daily have datetime indices
df_daily.index = pd.to_datetime(df_daily.index)
fact3_daily.index = pd.to_datetime(fact3_daily.index)

# Create an empty DataFrame for storing betas
betas = pd.DataFrame(index=indices, columns=df_daily.columns)

# Compute rolling betas
for industry in tqdm(df_daily.columns):
    industry_returns = df_daily[industry]

    # Ensure alignment with factor data
    merged_data = pd.concat([industry_returns, fact3_daily[["Mkt-RF", "RF"]]], axis=1).dropna()

    # Compute rolling regressions
    rolling_window = 12  # Approx. 12 months
    rolling_betas = []

    for end_date in indices[12:]:  # Skip first 12 months
        start_date = (end_date -  pd.DateOffset(months=12)) + pd.Timedelta(days=1)


        # Extract data for the rolling window
        window_data = merged_data.loc[start_date:end_date]
        if len(window_data) < 100:  # Ensure sufficient data points
            rolling_betas.append(np.nan)
            continue

        X = sm.add_constant(window_data["Mkt-RF"])
        y = window_data[industry] - window_data["RF"]
        model = sm.OLS(y, X).fit()

        rolling_betas.append(model.params["Mkt-RF"])

    # Assign computed betas to DataFrame
    betas.loc[indices[12:], industry] = rolling_betas

# Convert month-end indices to month-start
betas.index = betas.index.to_period('M').to_timestamp()

100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:58<00:00,  1.23s/it]


# Question 5

In [93]:
# Define the monthly date range
indices = pd.date_range(df_daily.index.min(), df_daily.index.max(), freq="M")

# Ensure df_daily and fact3_daily have datetime indices
df_daily.index = pd.to_datetime(df_daily.index)
fact3_daily.index = pd.to_datetime(fact3_daily.index)

# Create an empty DataFrame for storing idiosyncratic volatility
idiosyncratic_volatility = pd.DataFrame(index=indices, columns=df_daily.columns)

# Compute idiosyncratic volatility for each month
for date in tqdm(indices):
    # Extract data for the specific month
    start_date = date - pd.DateOffset(months=1) + pd.Timedelta(days=1)
    end_date = date

    # Filter data for the specific month
    ret_data = df_daily[(df_daily.index >= start_date) & (df_daily.index <= end_date)]
    factors_data = fact3_daily[(fact3_daily.index >= start_date) & (fact3_daily.index <= end_date)]

    # Calculate excess returns for each industry
    ret_data_excess = ret_data.subtract(factors_data['RF'], axis=0)

    # Calculate the idiosyncratic volatility for each industry
    for industry in ret_data.columns:
        # Prepare data for regression
        y = ret_data_excess[industry].values
        X = factors_data[['Mkt-RF', 'SMB', 'HML']].values
        X = sm.add_constant(X)

        # Fit the regression model
        model = sm.OLS(y, X).fit()

        # Compute standard deviation of residuals (idiosyncratic volatility)
        idio_vol = np.std(model.resid)

        # Assign computed idiosyncratic volatility to DataFrame
        idiosyncratic_volatility.at[date, industry] = idio_vol

# Convert month-end indices to month-start
idiosyncratic_volatility.index = idiosyncratic_volatility.index.to_period('M').to_timestamp()

100%|██████████████████████████████████████████████████████████████████████████████| 1182/1182 [00:27<00:00, 43.08it/s]


# Long-Short Portfolios

In [81]:
# Define the monthly date range for the characteristics DataFrame
indices = pd.date_range(start='1926-01-01', end='2024-12-01', freq='MS')

# Ensure all individual characteristics DataFrames have the same date range
df_mktcap = df_mktcap.reindex(indices).ffill()
be_me_monthly = be_me_monthly.reindex(indices).ffill()
momentum = momentum.reindex(indices).ffill()
betas = betas.reindex(indices).ffill()
idiosyncratic_volatility = idiosyncratic_volatility.reindex(indices).ffill()


dates = df.index  # df_monthly contains monthly industry returns

# Initialize DataFrames to store long-short portfolio returns
long_short_ew = pd.DataFrame(index=dates, columns=["Market Cap", "B/M", "Momentum", "Beta", "Volatility"])
long_short_vw = pd.DataFrame(index=dates, columns=["Market Cap", "B/M", "Momentum", "Beta", "Volatility"])

# Loop through each month t
for t in tqdm(range(len(dates) - 1)):  
    date = dates[t]
    next_month = dates[t + 1] 

    # Extract industry characteristics for time t
    characteristics = {
        "Market Cap": df_mktcap.loc[date],
        "B/M": be_me_monthly.loc[date],
        "Momentum": momentum.loc[date],
        "Beta": betas.loc[date],
        "Volatility": idiosyncratic_volatility.loc[date]
    }

    # Get next month's returns
    industry_returns = df.loc[next_month]

    # Process each characteristic separately
    for char_name, char_values in characteristics.items():
        # Drop NaN values before sorting
        char_values = char_values.dropna()


        # Sort industries based on the characteristic
        sorted_industries = char_values.sort_values()

        # Select top 5 (highest characteristic) and bottom 5 (lowest characteristic)
        top_5 = sorted_industries.index[-5:]
        bottom_5 = sorted_industries.index[:5]

        # Equal-Weighted (EW) Portfolio Returns
        ew_long = industry_returns[top_5].mean()
        ew_short = industry_returns[bottom_5].mean()
        long_short_ew.loc[next_month, char_name] = ew_long - ew_short
       

        # Value-Weighted (VW) Portfolio Returns
        mc_top = df_mktcap.loc[date, top_5]
        mc_bottom = df_mktcap.loc[date, bottom_5]

        # Weights
        top_weights = mc_top / mc_top.sum()
        bottom_weights = mc_bottom / mc_bottom.sum()

        vw_long = (industry_returns[top_5] * top_weights).sum()
        vw_short = (industry_returns[bottom_5] * bottom_weights).sum()
        long_short_vw.loc[next_month, char_name] = vw_long - vw_short
        
long_short_ew.columns = ["EW Market Cap", "EW B/M", "EW Momentum", "EW Beta", "EW Volatility"]
long_short_vw.columns = ["Val Market Cap", "Val B/M", "Val Momentum", "Val Beta", "Val Volatility"]
# Display results
print("Equal-Weighted Long-Short Returns:")
print(long_short_ew)

print("\nValue-Weighted Long-Short Returns:")
print(long_short_vw)


portfolio_results = pd.concat([long_short_ew, long_short_vw], axis = 1)
portfolio_results

100%|█████████████████████████████████████████████████████████████████████████████| 1181/1181 [00:07<00:00, 151.74it/s]

Equal-Weighted Long-Short Returns:
           EW Market Cap   EW B/M EW Momentum  EW Beta EW Volatility
1926-07-01           NaN      NaN         NaN      NaN           NaN
1926-08-01      -0.09772   0.2042         NaN      NaN       0.09644
1926-09-01       0.05342  0.01834         NaN      NaN      -0.02006
1926-10-01       0.03562  0.02408         NaN      NaN      -0.04962
1926-11-01      -0.00126 -0.02652         NaN      NaN       -0.0022
...                  ...      ...         ...      ...           ...
2024-08-01       0.07114 -0.03138    -0.02022 -0.06766      -0.03376
2024-09-01      -0.00896 -0.01078    -0.02518  0.03608       0.02912
2024-10-01       0.03996   0.0187    -0.06114  0.03504      -0.05298
2024-11-01      -0.08174  0.03668     0.03794   0.1582        0.0267
2024-12-01       0.07706 -0.05122    -0.00024  0.04702      -0.00476

[1182 rows x 5 columns]

Value-Weighted Long-Short Returns:
           Val Market Cap   Val B/M Val Momentum  Val Beta Val Volatility
19

,EW Market Cap,EW B/M,EW Momentum,EW Beta,EW Volatility,Val Market Cap,Val B/M,Val Momentum,Val Beta,Val Volatility
1926-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-08-01,-0.09772,0.2042,NaN,NaN,0.09644,-0.071482,0.127267,0.0,0.0,0.070455
1926-09-01,0.05342,0.01834,NaN,NaN,-0.02006,0.026141,0.028683,0.0,0.0,-0.002424
1926-10-01,0.03562,0.02408,NaN,NaN,-0.04962,0.015771,0.035838,0.0,0.0,-0.025072
1926-11-01,-0.00126,-0.02652,NaN,NaN,-0.0022,0.00665,-0.024773,0.0,0.0,0.01276
...,...,...,...,...,...,...,...,...,...,...
2024-08-01,0.07114,-0.03138,-0.02022,-0.06766,-0.03376,0.050458,-0.002261,0.022935,-0.002816,-0.050601
2024-09-01,-0.00896,-0.01078,-0.02518,0.03608,0.02912,0.003815,-0.031902,-0.079512,0.034669,0.088019
2024-10-01,0.03996,0.0187,-0.06114,0.03504,-0.05298,0.02337,0.039976,0.01759,0.058597,-0.040748
2024-11-01,-0.08174,0.03668,0.03794,0.1582,0.0267,-0.03847,0.089661,0.006141,0.046474,0.172016


# Performance Metrics

In [82]:
# Function to calculate annualized mean return
def mean_return(rets):
    return 12 * rets.mean()

# Function to calculate annualized Sharpe Ratio
def sharpe_ratio(rets, rf):
    excess_returns = rets - rf
    return np.sqrt(12) * excess_returns.mean() / rets.std()

# Function to calculate annualized alpha from Fama-French regression
def alpha(rets, ff):
    rets = pd.to_numeric(rets, errors="coerce")  # Ensure numeric
    ff = ff.apply(pd.to_numeric, errors="coerce")  # Ensure numeric
    ff = sm.add_constant(ff, has_constant="add")   # Add constant for regression
    df = pd.concat([rets, ff], axis=1).dropna()    # Drop NaN values
    if df.empty:
        return np.nan  # Return NaN if no valid data
    model = sm.OLS(df.iloc[:, 0], df.iloc[:, 1:]).fit()
    return 12 * model.params["const"]

# Function to calculate the p-value of alpha
def alpha_p_val(rets, ff):
    rets = pd.to_numeric(rets, errors="coerce")
    ff = ff.apply(pd.to_numeric, errors="coerce")
    ff = sm.add_constant(ff, has_constant="add")
    df = pd.concat([rets, ff], axis=1).dropna()
    if df.empty:
        return np.nan  # Return NaN if no valid data
    model = sm.OLS(df.iloc[:, 0], df.iloc[:, 1:]).fit()
    return model.pvalues["const"]

# Performance Metrics Output

In [94]:
# Define time periods
start_dates = [datetime(1950, 1, 1), datetime(1990, 1, 1), datetime(2000, 1, 1)]
results = {}

for start in start_dates:
    period_name = f"{start.year}-2024"

    # Select portfolio returns for the current period
    current_portfolio = portfolio_results.loc[portfolio_results.index >= start].dropna()


    # Select relevant factor models
    current_ff3 = fact3.loc[fact3.index >= start, ["Mkt-RF", "SMB", "HML", "RF"]]
    current_ff4 = fact4.loc[fact4.index >= start, ["Mkt-RF", "SMB", "HML", "Mom   ", "RF"]]
    current_ff5 = fact5.loc[fact5.index >= max(start, datetime(1963, 7, 1)), ["Mkt-RF", "SMB", "RMW", "CMA", "RF"]]

    # Compute mean return and Sharpe ratio
    mean_rets = current_portfolio.apply(mean_return)
    sharpe_ratios = current_portfolio.apply(lambda x: sharpe_ratio(x, current_ff3["RF"]))

    # Compute alphas and p-values for 3, 4, and 5 factor models
    alpha_ff3 = current_portfolio.apply(lambda x: alpha(x - current_ff3["RF"], current_ff3[["Mkt-RF", "SMB", "HML"]]))
    p_vals_ff3 = current_portfolio.apply(lambda x: alpha_p_val(x - current_ff3["RF"], current_ff3[["Mkt-RF", "SMB", "HML"]]))

    alpha_ff4 = current_portfolio.apply(lambda x: alpha(x - current_ff4["RF"], current_ff4[["Mkt-RF", "SMB", "HML", "Mom   "]]))
    p_vals_ff4 = current_portfolio.apply(lambda x: alpha_p_val(x - current_ff4["RF"], current_ff4[["Mkt-RF", "SMB", "HML"]]))

    alpha_ff5 = current_portfolio.loc[current_portfolio.index >= datetime(1963, 7, 1)].apply(lambda x: alpha(x - current_ff5["RF"], current_ff5[["Mkt-RF", "SMB", "RMW", "CMA"]]))
    p_vals_ff5 = current_portfolio.loc[current_portfolio.index >= datetime(1963, 7, 1)].apply(lambda x: alpha_p_val(x - current_ff5["RF"], current_ff5[["Mkt-RF", "SMB", "RMW", "CMA"]]))

    # Compile results
    results_df = pd.concat([mean_rets, sharpe_ratios, alpha_ff3, p_vals_ff3, alpha_ff4, p_vals_ff4, alpha_ff5, p_vals_ff5], axis=1)
    results_df.columns = [
        "Mean Return", "Sharpe Ratio",
        "Alpha 3F", "P-Value Alpha 3F",
        "Alpha 4F", "P-Value Alpha 4F",
        "Alpha 5F", "P-Value Alpha 5F"
    ]
    print(period_name)
    print(results_df)


1950-2024
                Mean Return  Sharpe Ratio  Alpha 3F  P-Value Alpha 3F  \
EW Market Cap     -0.023079     -0.395685 -0.025994      8.475204e-02   
EW B/M             0.018304     -0.137578 -0.071109      1.951267e-07   
EW Momentum        0.122508      0.429112  0.106120      2.681076e-06   
EW Beta            0.016764     -0.115584 -0.068669      2.115977e-04   
EW Volatility     -0.012314     -0.363467 -0.066084      4.866306e-05   
Val Market Cap    -0.030365     -0.462922 -0.037443      1.043314e-02   
Val B/M            0.006617     -0.201162 -0.084210      1.270218e-10   
Val Momentum       0.111014      0.345950  0.091736      1.514365e-04   
Val Beta           0.019885     -0.094643 -0.063827      8.909382e-04   
Val Volatility    -0.007418     -0.269942 -0.066480      8.023762e-04   

                Alpha 4F  P-Value Alpha 4F  Alpha 5F  P-Value Alpha 5F  
EW Market Cap  -0.001010      3.746751e-01 -0.036114          0.203740  
EW B/M         -0.005368      1.907118e-